In [1]:
import torch
import pandas as pd
import re
import csv
import numpy as np
import torchtext
from collections import defaultdict
from torchtext import data
import nltk
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from nltk import sent_tokenize,word_tokenize
from torchtext import data,vocab
from tqdm.notebook import tqdm, tqdm_notebook,tnrange
from sklearn.metrics import accuracy_score
from torchsummary import summary

In [2]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sanchit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def tokenize(s):
    return re.findall(r"[\w']+|[.,!?;]",s)

In [4]:
featstrain = {}
featstest = {}
import csv
with open ('trainOPN.csv',newline = '') as file:
  read = csv.reader(file,delimiter = ',') 
  for line in read:
    # print(line)
    temp = line[2].split(' ')
    temp = [float(f) for f in temp]
    featstrain[int(line[3])] = temp

with open ('testOPN.csv',newline = '') as file:
  read = csv.reader(file,delimiter = ',')
  for line in read:
    temp = line[2].split(' ')
    temp = [float(f) for f in temp]
    featstest[int(line[3])] = temp
# print((featstest[322]))

In [5]:
txt_field = data.Field(sequential = True,tokenize = tokenize,include_lengths = False, use_vocab = True)
label_field = data.Field(sequential = False,use_vocab=False,pad_token=None,unk_token=None)
vec_field = data.Field(sequential = False,use_vocab =False , pad_token= None, unk_token = None)
train_val_fields = [('EssayText',txt_field),('Personality',label_field),('Mariesse',None),('idx',vec_field)]

In [6]:
vec = vocab.Vectors('glove.6B.300d.txt','./')

In [7]:
train,val = data.TabularDataset.splits(path = './',format = 'csv',train='trainOPN.csv',validation = 'testOPN.csv',fields = train_val_fields)


In [8]:
txt_field.build_vocab(train,val,vectors = vec)

In [9]:
label_field.build_vocab(train)
# vec_field.build_vocab(train)
# print(txt_field.vocab.vectors.shape)
print((txt_field).vocab['!'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

64


In [10]:
traindl,valdl = data.BucketIterator.splits(datasets=(train,val),batch_size = 16,device = device,sort_key = lambda x : len(x.EssayText),sort_within_batch = False)

In [23]:
batch = next(iter(traindl))
# print(dir(batch))
print((batch.EssayText[:,0]))
test_list=(batch.EssayText[:,0]).tolist()
print(len(test_list))
def text_to_sent(test_list):
    size = len(test_list) 
    idx_list = [idx + 1 for idx, val in
            enumerate(test_list) if val == 2 or val == 55 or val == 64 ]

    temp_li_final = [test_list[i: j] for i, j in
        zip([0] + idx_list, idx_list + 
        ([size] if idx_list[-1] != size else []))] 
    return temp_li_final
temp_li_final = text_to_sent(test_list)
    
# temp_sent2 = []
# for i in temp_li_final:
#     temp_sent = ''
#     for x in (i):
#         word = txt_field.vocab.itos[x]
#         temp_sent += word + ' '
#     temp_sent2.append(temp_sent)

# print((temp_sent2))


tensor([ 342, 1191,    3,  ...,    1,    1,    1], device='cuda:0')
1388


In [17]:
class CNN1d(nn.Module):
    def __init__(self,vocab_size,embedding_dim,pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim,padding_idx = pad_idx)
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = embedding_dim,out_channels = 50,kernel_size = fs) for fs in (1,2,3)])
        self.conv2 = nn.Conv1d(in_channels = 50,out_channels = 100,kernel_size = 2)
        self.fc1 = nn.Linear(224884,50) #Change this 
        self.fc2 = nn.Linear(50,1)
        self.dropout = nn.Dropout(0.5)
    def forward(self,text,len_sentences):
        print("text",text.size())
        embedded = self.embedding(text.T)
        embedded = embedded.permute(0,2,1)
        print(embedded.size(),"embedded")
        # mariesse = [float(f) for f in mariesse]
#         mariesse = torch.from_numpy(mariesse).float()
#         mariesse = torch.tensor(mariesse,device = device,dtype = embedded.dtype)
        # print(mariesse.size(),"mariesse")
        # mariesse = torch.FloatTensor(mariesse,dtype = embedded,device = embedded.device)
        x=embedded.size(2)
        y=3000-x
        # print(y,"hello")
        batch_size=embedded.size(0)
        # z=np.zeros((batch_size,100,y,dtype = embedded.dtype,device = embedded.device))
        z = torch.zeros(batch_size,300,y,dtype = embedded.dtype,device = embedded.device)
        # z1=torch.from_numpy(z).float()
        lz=[embedded,z]
        # print(type(embedded))
        zcat = torch.cat(lz, dim = 2,)
        print("zcat",zcat.size())
        conved = [F.relu(conv(zcat)) for conv in self.convs]
        print("Conved sizes below")
        for c in conved:
          print(c.size())
        pooled = [F.max_pool1d(conv,(2)) for conv in conved]
        print("Pool Sizes below")
        for pl in pooled:
            print(pl.size())
        cat = torch.cat(pooled,dim = 2)
        print("cat after layer 1",cat.size())
        conved2 = F.relu(self.conv2(cat))
        print("conved2",conved2.size())
        pooled2 = F.max_pool1d(conved2,2)
        print(pooled2.size(),"pooled after layer 1")
        pooled2 = pooled2.reshape(batch_size,224800)
        # print(pooled2.size(),"size2")
        # mairesse_layer = torch.from_numpy(matrix_mair).float()
        # mairesse_layer.device = pooled2.device
#         concat_mairesse = torch.cat([pooled2,mariesse],dim = 1)
        # print(concat_mairesse.size(),"concat")
        # print(pooled2.size(),"pooled2")
        full1 = self.fc1(pooled2)
        final1 = self.dropout(full1)
        full2 = self.fc2(final1)
        # final = self.dropout(full2)
        return full2
#         return pooled2

In [ ]:
input_dim = len(txt_field.vocab)
embedding_dim = 300
pad_idx = txt_field.vocab.stoi[txt_field.pad_token]
model = CNN1d(input_dim,embedding_dim,pad_idx)
print(model)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

ex = next(iter(traindl))
print(ex.batch_size)

In [ ]:
pretrained_embeddings = txt_field.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
unk_idx = txt_field.vocab.stoi[txt_field.unk_token]
model.embedding.weight.data[unk_idx] = torch.zeros(300)
model.embedding.weight.data[pad_idx] = torch.zeros(300)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model,iterator,optimizer,criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        mariesse_vecs = np.zeros((batch.batch_size,84))
#         j = 0
#         temp_idx = batch.idx.tolist()
#         temp_idx = [int(i) for i in temp_idx]
#         for i in temp_idx:
#           mariesse_vecs[j] = featstrain[i]
#           j += 1
        for idx in range(len(batch.batch_size)):
            test_list=(batch.EssayText[:,idx]).tolist()
            temp_li_final = text_to_sent(test_list)
            preds = []
            for sent in temp_li_final:
                predictions = model(sent).squeeze(1)
                preds.append(predictions)
            preds
            
        batch.Personality = batch.Personality.type_as(predictions)
        loss = criterion(predictions,batch.Personality)
        acc = binary_accuracy(predictions,batch.Personality)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss/len(iterator),epoch_acc/len(iterator)

In [ ]:
def evaluate(model,iterator,criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            mariesse_vecs = np.zeros((batch.batch_size,84))
            j = 0
            temp_idx = batch.idx.tolist()
            temp_idx = [int(i) for i in temp_idx]
            for i in temp_idx:
              mariesse_vecs[j] = featstest[i]
              j += 1 
            predictions = model(batch.EssayText,mariesse_vecs).squeeze(1)
            batch.Personality = batch.Personality.type_as(predictions)
            loss = criterion(predictions,batch.Personality)
            acc = binary_accuracy(predictions,batch.Personality)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss/len(iterator),epoch_acc/len(iterator)        

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 1

best_valid_loss = float('inf')
train_loss_list = []
valid_loss_list = []
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, traindl, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valdl, criterion)
    train_loss_list.append(train_loss)
    valid_loss_list.append(valid_loss)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss_list)
plt.plot(valid_loss_list)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'],loc = 'upper left')
plt.show()